Checking GPU

In [ ]:
!nvidia-smi

Install all nescessary library

In [ ]:
!pip install -q git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

In [ ]:
!pip install torchvision

Import library

In [ ]:
import os
import cv2
import numpy as np
import supervision as sv
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

Set Home directory

In [ ]:
HOME = os.getcwd()
print("HOME:", HOME)

Define download from url function and download SAM weights

In [ ]:
import requests
import os

def download_file(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"File downloaded successfully: {save_path}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")


url = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'

save_path = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")

download_file(url, save_path)


In [ ]:
CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
print(DEVICE)

Set the SAM model with the downloaded weight on GPU

In [ ]:
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_predictor = SamPredictor(sam)

Set the box for masking

In [ ]:
default_box = {'x': 132, 'y': 151, 'width': 300, 'height': 300, 'label': ''}

box = default_box
box = np.array([
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
])

In [ ]:
source_dir = os.path.join(HOME, 'Original/train/')
os.makedirs(source_dir, exist_ok=True)

Mask the WBC in the middle of the image and download the segmented image

In [ ]:
import os
import cv2
import numpy as np

source_dir = os.path.join(HOME, 'Original/train/')
target_root_dir = os.path.join(HOME, 'Original_segmentation/train/')

os.makedirs(target_root_dir, exist_ok=True)

for folder_name in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder_name)

    target_folder_path = os.path.join(target_root_dir, folder_name)
    os.makedirs(target_folder_path, exist_ok=True)

    for image_filename in os.listdir(folder_path):
        IMAGE_PATH = os.path.join(folder_path, image_filename)
        image_bgr = cv2.imread(IMAGE_PATH)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        mask_predictor.set_image(image_rgb)

        masks, scores, logits = mask_predictor.predict(
            box=box,
            multimask_output=True
        )

        mask = masks[np.argmax(scores)]
        mask = mask.astype(bool)
        masked_image = np.zeros_like(image_rgb)

        for c in range(3):
            masked_image[:, :, c] = image_rgb[:, :, c] * mask

        y_indices, x_indices = np.where(mask)
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()

        cropped_image = masked_image[y_min:y_max, x_min:x_max]

        cropped_image_path = os.path.join(target_folder_path, image_filename + "_segment.png")
        cv2.imwrite(cropped_image_path, cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))
